In [ ]:
import tensorflow as tf
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from keras.utils.vis_utils import plot_model
import numpy as np
import codecs
import random
import string
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from tensorflow.keras import layers
import tensorflow.keras as keras
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### **Read en,fa Data**

In [ ]:
%%time
with open("TEP-fa.txt") as f:
    fa_file = f.read() 

with open("TEP-en.txt") as f:
    en_file = f.read()    

CPU times: user 211 ms, sys: 109 ms, total: 320 ms
Wall time: 319 ms


In [ ]:
fa_data = fa_file.strip().split('\n')
en_data = en_file.strip().split('\n')

In [ ]:
len(fa_data)

612086

In [ ]:
len(en_data)

612086

In [ ]:
data = pd.DataFrame(columns=['en','fa'])
data['fa'] = fa_data
data['en'] = en_data

In [ ]:
data.head()

,en,fa
0,raspy breathing .,صداي خر خر .
1,dad .,پدر .
2,maybe its the wind .,شايد صداي باد باشه .
3,no .,نه .
4,stop please stop .,دست نگه داريد خواهش ميکنم دست نگه داريد .


### **Export DataFrame to CSV**

In [ ]:
data.to_csv('en-fa_MT_dataset.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r "/content/en-fa_MT_dataset.csv" "/content/drive/MyDrive/en-fa_MT_dataset.csv"

Mounted at /content/drive


# **Read Dataset**

In [ ]:
!gdown --id 101S8yZESRK5YL0a886tTz1hnCFzhaRxe

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=101S8yZESRK5YL0a886tTz1hnCFzhaRxe
To: /content/en-fa_MT_dataset.csv
100% 55.4M/55.4M [00:00<00:00, 298MB/s]


In [ ]:
data = pd.read_csv('en-fa_MT_dataset.csv')

In [ ]:
data.head(3)

,en,fa
0,raspy breathing .,صداي خر خر .
1,dad .,پدر .
2,maybe its the wind .,شايد صداي باد باشه .


In [ ]:
len(data)

612086

In [ ]:
data['en_size'] = data['en'].str.count(' ')
data['fa_size'] = data['fa'].str.count(' ')

In [ ]:
data.head()

,en,fa,en_size,fa_size
0,raspy breathing .,صداي خر خر .,2,3
1,dad .,پدر .,1,1
2,maybe its the wind .,شايد صداي باد باشه .,4,4
3,no .,نه .,1,1
4,stop please stop .,دست نگه داريد خواهش ميکنم دست نگه داريد .,3,8


In [ ]:
data['en_no_punctuation'] = data['en'].str.replace('[^\w\s]','')
data['en_no_punctuation'] = data["en_no_punctuation"].str.lower() 

In [ ]:
data['fa_no_punctuation'] = '<start> ' + data['fa'].str.replace('[^\w\s]','') + ' <end>'

In [ ]:
data.head()

,en,fa,en_size,fa_size,en_no_punctuation,fa_no_punctuation
0,raspy breathing .,صداي خر خر .,2,3,raspy breathing,<start> صداي خر خر <end>
1,dad .,پدر .,1,1,dad,<start> پدر <end>
2,maybe its the wind .,شايد صداي باد باشه .,4,4,maybe its the wind,<start> شايد صداي باد باشه <end>
3,no .,نه .,1,1,no,<start> نه <end>
4,stop please stop .,دست نگه داريد خواهش ميکنم دست نگه داريد .,3,8,stop please stop,<start> دست نگه داريد خواهش ميکنم دست نگه داري...


In [ ]:
en_data = data['en_no_punctuation'].values
fa_data = data['fa_no_punctuation'].values

In [ ]:
en_data = [re.sub('\s+', ' ', str(sentence)) for sentence in en_data]
fa_data = [re.sub('\s+', ' ', str(sentence)) for sentence in fa_data]

In [ ]:
text_pairs = []
for i in range(len(en_data)):
  en = en_data[i]
  fa = fa_data[i]
  text_pairs.append((en, fa))

In [ ]:
text_pairs[0:5]

[('raspy breathing ', '<start> صداي خر خر <end>'),
 ('dad ', '<start> پدر <end>'),
 ('maybe its the wind ', '<start> شايد صداي باد باشه <end>'),
 ('no ', '<start> نه <end>'),
 ('stop please stop ',
  '<start> دست نگه داريد خواهش ميکنم دست نگه داريد <end>')]

In [ ]:
len(text_pairs)

612086

In [ ]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

612086 total pairs
428462 training pairs
91812 validation pairs
91812 test pairs


In [ ]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 512


def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


en_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
fa_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]
en_vectorization.adapt(train_eng_texts)
fa_vectorization.adapt(train_spa_texts)

In [ ]:
def format_dataset(en, fa):
    en = en_vectorization(en)
    fa = fa_vectorization(fa)
    return (
        {
            "encoder_inputs": en,
            "decoder_inputs": fa[:, :-1],
        },
        fa[:, 1:],
    )

In [ ]:
def make_dataset(pairs):
    en_texts, fa_texts = zip(*pairs)
    en_texts = list(en_texts)
    fa_texts = list(fa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((en_texts, fa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()

In [ ]:
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [ ]:
train_ds

<CacheDataset element_spec=({'encoder_inputs': TensorSpec(shape=(None, 20), dtype=tf.int64, name=None), 'decoder_inputs': TensorSpec(shape=(None, 20), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, 20), dtype=tf.int64, name=None))>

In [ ]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (512, 20)
inputs["decoder_inputs"].shape: (512, 20)
targets.shape: (512, 20)


In [ ]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


In [ ]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(latent_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

In [ ]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [ ]:
transformer.summary()

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 positional_embedding (Position  (None, None, 256)   3845120     ['encoder_inputs[0][0]']         
 alEmbedding)                                                                                     
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder (Transform  (None, None, 256)   3155456     ['positional_embedding[

In [ ]:
transformer.compile("rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
epochs = 30
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds, batch_size=128)

Epoch 1/30
837/837 [==============================] - 352s 412ms/step - loss: 1.9576 - accuracy: 0.3400 - val_loss: 1.6008 - val_accuracy: 0.4100
Epoch 2/30
837/837 [==============================] - 344s 411ms/step - loss: 1.6248 - accuracy: 0.4120 - val_loss: 1.4817 - val_accuracy: 0.4368
Epoch 3/30
837/837 [==============================] - 344s 411ms/step - loss: 1.5213 - accuracy: 0.4378 - val_loss: 1.4035 - val_accuracy: 0.4546
Epoch 4/30
837/837 [==============================] - 344s 411ms/step - loss: 1.4575 - accuracy: 0.4545 - val_loss: 1.3735 - val_accuracy: 0.4618
Epoch 5/30
837/837 [==============================] - 344s 411ms/step - loss: 1.4088 - accuracy: 0.4672 - val_loss: 1.3618 - val_accuracy: 0.4672
Epoch 6/30
837/837 [==============================] - 344s 411ms/step - loss: 1.3701 - accuracy: 0.4779 - val_loss: 1.3581 - val_accuracy: 0.4692
Epoch 7/30
837/837 [==============================] - 343s 410ms/step - loss: 1.3353 - accuracy: 0.4874 - val_loss: 1.3326 -

In [ ]:
fa_vocab = fa_vectorization.get_vocabulary()
fa_index_lookup = dict(zip(range(len(fa_vocab)), fa_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = en_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = fa_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = fa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence




NameError: ignored

In [ ]:
test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(30):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequence(input_sentence)
    print(input_sentence,translated)

go around  <start> بيا بيا end    end     end      end end 
axe gang my ass  <start> دو دو هاي را end       end  را    end end 
there was nothing that ever scared you nothin  <start> اون اين به اين يک يک تو را end     end     end 
get that you get it  <start> تو چي end    که     end      end end 
for starters i had to cover your ass i didnt ask you to  <start> براي من من من را را را را را را را را را را را را را را را را
they see the wealth of lionath on offer that seems more real than any alliance  <start> اونا يک يک يک از يک را را را را را را را را را را end  end 
well find him just hang in there miss and we will get to you  <start> حالا چي که بايد را را را را را را را را را را کنيم end   کنيم 
true to our ways and you will always be worthy of this  <start> اونا يک يک ما را را را را را را را end  را    را end 
you dont know what its like to be afraid of your powers afraid to get close to anybody  <start> تو فکر را بايد را را را را را را را را را را را را را را را را
daylight deals a 